In [ ]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split # Example import



####Placeholder Data (REPLACE WITH YOUR ACTUAL DATA).

X_train_df = pd.DataFrame(np.random.rand(100, 10))
X_val_df = pd.DataFrame(np.random.rand(50, 10))
X_test_df = pd.DataFrame(np.random.rand(50, 10))

# True class labels
y_train_df = pd.Series(np.random.randint(0, 3, 100))
y_val_df = pd.Series(np.random.randint(0, 3, 50))
y_test_df = pd.Series(np.random.randint(0, 3, 50))

# Logits from the teacher model for a 3-class problem. It will have 3 columns.
all_train_logits_df = pd.DataFrame(np.random.rand(100, 3))


def softmax(logits):

    e_logits = np.exp(logits - np.max(logits, axis=1, keepdims=True))
    return e_logits / e_logits.sum(axis=1, keepdims=True)


def run_distillation_experiment(random_state, X_train, y_train, X_val, y_val, X_test, y_test, train_logits):

    ##Trains a set of regressors to mimic teacher logits and evaluates the resulting student classifier.

    models = []

    ## Train one regressor for each logit column
    for i in range(train_logits.shape[1]):
        model = lgb.LGBMRegressor(
            device_type='gpu', # Change to 'cpu' if no GPU
            n_estimators=6,
            num_leaves=50,
            max_depth=12,
            learning_rate=0.3,
            random_state=random_state
        )
        # Fit the model to predict the i-th logit
        model.fit(X_train, train_logits.iloc[:, i])
        models.append(model)

    ## Predict logits for all datasets
    pred_logits_train = np.array([m.predict(X_train) for m in models]).T
    pred_logits_val = np.array([m.predict(X_val) for m in models]).T
    pred_logits_test = np.array([m.predict(X_test) for m in models]).T

    ## Convert predicted logits to class labels via softmax
    class_labels_train = np.argmax(softmax(pred_logits_train), axis=1)
    class_labels_val = np.argmax(softmax(pred_logits_val), axis=1)
    class_labels_test = np.argmax(softmax(pred_logits_test), axis=1)

    ## Calculate performance metrics
    acc_train = accuracy_score(y_train, class_labels_train)
    acc_val = accuracy_score(y_val, class_labels_val)
    acc_test = accuracy_score(y_test, class_labels_test)

    f1_train = f1_score(y_train, class_labels_train, average='weighted')
    f1_val = f1_score(y_val, class_labels_val, average='weighted')
    f1_test = f1_score(y_test, class_labels_test, average='weighted')

    # Print the confusion matrix for this run for inspection. This step can be done if class wise performance needs to be inspected
    print(f"Confusion Matrix for seed {random_state}:\n{confusion_matrix(y_test, class_labels_test)}\n")

    return acc_train, acc_val, acc_test, f1_train, f1_val, f1_test


##Running the experiment with multiple random seeds##
results = []
seeds = [20, 42, 100]
for seed in seeds:
    exp_results = run_distillation_experiment(
        seed,
        X_train_df, y_train_df,
        X_val_df, y_val_df,
        X_test_df, y_test_df,
        all_train_logits_df
    )
    results.append(exp_results)

## Aggregate and print final results, compute std if required#
results_np = np.array(results)
mean_results = results_np.mean(axis=0)


print("--- Final Aggregated Results ---")
print(f"Mean Accuracy (Train | Val | Test): {mean_results[0]:.4f} | {mean_results[1]:.4f} | {mean_results[2]:.4f}")
)

print(f"Mean F1-Score (Train | Val | Test): {mean_results[3]:.4f} | {mean_results[4]:.4f} | {mean_results[5]:.4f}")
